In [1]:
%%capture
# run this cell if this package is not installed
# !pip install nbformat

In [1]:
%%capture
%run setup.ipynb

In [2]:
# Define a function to remove p1 and p2
def remove_start_end(text):
    
    p1 = "Chat Conversation Start"
    p2 = "Chat Conversation End"
    
    # Remove p1 only the first time it appears
    text = re.sub(rf'^\s*{re.escape(p1)}\s*', '', text)

    # Remove p2 only the last time it appears
    text = re.sub(rf'\s*{re.escape(p2)}\s*$', '', text)

    return text

# Apply the function to the "Content" column
df['Content'] = df['Content'].apply(remove_start_end)

In [45]:
def normalize_document(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # remove email addresses
    doc = re.sub(r'\b\S*@\S*\.\S*\b', '', doc)
    # lowercase and remove special characters\whitespace
    doc = re.sub(r'[^a-zA-Z\s]', '', str(doc), re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
            
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc


In [46]:
df['Content'].apply(normalize_document)

0        u good girls bed haha omg miss lot lovely frie...
1        u hi u great thanks kind fine think problem as...
2        u hello u hello hello best friend finally real...
3        u want ask study lot mathematics business engi...
4        u hi checked courses ask friends see one took ...
                               ...                        
11331    robert weiss subject apr gods promise philippi...
11332    kent sandvik subject apr gods promise john org...
11333    kent sandvik subject disillusioned protestant ...
11334    cutter subject biblical backing koreshs tape c...
11335    subject albert sabin rich fox univ south dakot...
Name: Content, Length: 11336, dtype: object